In this notebook, you'll implement object detection using the very power YOLO (You Only Look Once) model.   

Many of the ideas in this notebook are described in the two YOLO papers: [Redmon et al., 2016](https://arxiv.org/abs/1506.02640) and [Redmon and Farhadi, 2016](https://arxiv.org/abs/1612.08242). 

**By the end of this, you will be able to:**
-  Detect objects in a car detection dataset
-  Implement non-max suppression to increase accuracy
-  Implement intersection over union (IoU) to evaluate bounding box predictions
-  Handle bounding boxes, a type of image annotation popular in deep learning.

### Package 


In [3]:
import os 

In [ ]:
branch_path = 'Chapter04-Convolutional-Neural-Networks/Object_Detection/W3A1'
os.chdir(os.path.join(os.getcwd(), branch_path))
os.getcwd()

In [ ]:
import argparse

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import ImageFont, ImageDraw, Image

import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor

from tensorflow.keras.models import load_model

from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image

%matplotlib inline

### 1 - Problem Statement

You are working on a self-driving car. Go you! As a critical component of this project, you'd like to first build a car detection system. To collect data, you've mounted a camera to the hood (meaning the front) of the car, which takes pictures of the road ahead every few seconds as you drive around. 

<center>
<video width="400" height="200" src="nb_images/road_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>

<caption><center> Pictures taken from a car-mounted camera while driving around Silicon Valley. <br> Dataset provided by <a href="https://www.drive.ai/">drive.ai</a>.
</center></caption>

You've gathered all these images into a folder and labelled them by drawing bounding boxes around every car you found. Here is an example of what your bounding boxes look like:

<center>
<img src="nb_images/box_label.png" style="width:500px;height:250;">
<caption><center> <u><b>Figure 1</u></b>: Definition of a box<br> </center></caption>
</center>

If there are 80 classes you want the object detector to recgonize, you can represent the class label c either as an integer from 1 to 80, or as an 80-dimensional vector (with 80 numbers) one component of which is 1 and the rest are all 0. The video lectures used the latter representation;  in this notebook, you'll use both representations, depending on which is more convenient for a particular step.  

Because the YOLO model is very computationally expensive to train, the pre-trained weights are already loaded for you to use. 



## 2 - YOLO

"You Only Look Once" (YOLO) is a popular algorithm because it achieves high accuracy while also being able to run in real-time.  This algorithm "only look once" at the image in the sense that it requires only one forward propagation pass through the network to make predictions.  After non-max suppression, it then outputs recgnized objects together with the bounding boxes.

### 2.1 - Model Details

#### Inputs and outputs
- The **input** is a batch of images, and each image has the shape (608, 608, 3).
- The **output** is a list of bounding boxes along with the recgnized classes. Each bounding box is represented by 6 numbers: $(p_c, b_x, b_y, b_h, b_w, c)$ as explained in Figure 1 above. If you expand $c$ into an 80-dimensional one-hot vector, each bounding box is represented by 85 numbers.

#### Anchor boxes
* Anchor boxes are chosen by exploring the training data to choose reasonable height/width ratios that represent the different classes.  For this assignment, 5 anchor boxes were chosen for you (to conver 80 classes), and stored in the file `./model_data/yolo_anchors.txt`.
* The YOLO architecture is: IMAGE (m, 608, 608, 3) --> DEEP CNN --> ENCODING (m, 19, 19, 5, 85). 
- 19 x 19 comes from the fact that the input image is divided into a 19 x 19 grid
- 5 is the number of anchor boxes
- 85 is from the 85 numbers representing each bounding box: $(p_c, b_x, b_y, b_h, b_w, c)$ where $c$ is an 80-dimensional one-hot vector.

#### Encoding
Let's look in greater detail at what this encoding represents. 

<img src="nb_images/architecture.png" style="width:700px;height:400;">
<caption><center> <u><b> Figure 2 </u></b>: Encoding architecture for YOLO<br> </center></caption>

If the center/midpoint of an object falls into a grid cell, that grid cell is responsible for detecting object.  

Since you're using 5 anchor boxes, each of the 19 x 19 thus encodes inforamtion about 5 boxes. Anchor boxes are defined only by their height and width. 

For simplicity, you'll flatten the last two dimensions of the shape (19, 19, 5, 85) into a single dimension of size 5 x 85 = 425. Thus the output of YOLO is of shape (19, 19, 425).   


#### Class score
Now, for each box (of each cell), you'll compute the following element-wise product and extract a probability that the box contains a certain class.

The class score is $score_{c,i} = p_{c} \times c_{i}$: the probability that there is an object $p_{c}$ times the probability that the object is a certain class $c_{i}$.

*Example of figure 4*
- In figure 4, let's say for box 1 (cell 1), the probability that an object exists is $p_1 = 0.60$. So there is a 60% chance that an object exists in box 1 of cell 1.
- The probability that the object is the class "category 3(a car)" is $c_3 = 0.73$. 
- The score for box 1 and for category 3 is score_{3,1} = p_1 * c_3 = 0.60 * 0.73 = 0.438.
- Let's say you calculate the score for all 80 classes in box 1, and find that the score for the car is the maximum. Then you would predict that box 1 contains a car, with a confidence of 43.8%.


#### Visualizing classes 
Here is one way to visualize what YOLO is predicting on an image:

- For each of the 19 x 19 grid cells, find the maximum of the probility scores 
- Color that grid cell according to what object that grid cell considers the most likely.  

<center>
<img src="nb_images/proba_map.png" style="width:300px;height:300;">
<caption><center> <u><b>Figure 5</u></b>: Each one of the 19x19 grid cells is colored according to which class has the largest predicted probability in that cell.<br> </center></caption>

</center>


Note that this visualization isn't a core part of the YOLO algorithm itself for making predictions; it's just a nice way of visualizing an intermediate result of the algorithm.


#### Visualizing bounding boxes
Another way to visualize YOLO's outputs is to plot the bounding boxes that it outputs. Doing that results in a visualization like this: 

<center>
<img src="nb_images/anchor_map.png" style="width:200px;height:200;">
<caption><center> <u><b>Figure 6</u></b>: Each cell gives you 5 boxes. In total, the model predicts: 19x19x5 = 1805 boxes just by looking once at the image (one forward pass through the network)! Different colors denote different classes. <br> </center></caption>
</center>

#### Non-max suppression
In the figure above, the only boxes plotted are ones for which the model has assigned a high probability, but this is still too many boxes.  You'd like to reduce the algorithm's output to a much smaller number of detected objects.

To do so, you'll use non-max suppresion (NMS). Specifically, you'll carry out these steps:
- Get rid of boxes with a low score. Meaning, the box is not very confident about detecting a class, either due to the low probability of any object, or low probability of this particular class
- Select only one box when several boxes overlap with each other and detect the same object.





#### 2.2 Filtering with a Threshold on Class Scores

You're going to first apply a filter by thresholding, meaning you'll get rid of any box for which the classes score is less than a chosen threshold.

The model gives you a total of 19 x 19 x 5 x 85 numbers, with each box described by 85 numbers.   It's convenient to rearrange the (19,19,5,85) (or (19,19,425)) dimensional tensor into the following variables:
- `box_confidence`: tensor of shape $(19, 19, 5, 1)$ containing $p_c$ (confidence probability that there's some object) for each of the 5 boxes predicted in each of the 19x19 cells.
- `boxes`: tensor of shape $(19, 19, 5, 4)$ containing the midpoint and dimensions $(b_x, b_y, b_h, b_w)$ for each of the 5 boxes in each cell.
- `box_class_probs`: tensor of shape $(19, 19, 5, 80)$ containing the "class probabilities" $(c_1, c_2, ... c_{80})$ for each of the 80 classes for each of the 5 boxes per cell.

### Exercise 1 - yolo_filter_boxes

Implement `yolo_filter_boxes`

1, Compute box scores by doing the element-wise product of `box_confidence` and `box_class_probs`.
the Following code may help you choose the right operator:

```python
a = np.random.randn(19, 19, 5, 1)
b = np.random.randn(19, 19, 5, 80)
c = a * b # shape of c will be (19, 19, 5, 80)
```

2. For each box, find:
- The index of the class with the maximum box score
- The corresponding box score
    
**Useful References**
    * [tf.math.argmax](https://www.tensorflow.org/versions/r2.3/api_docs/python/tf/math/argmax)
    * [tf.math.reduce_max](https://www.tensorflow.org/versions/r2.3/api_docs/python/tf/math/reduce_max)

**Helpful Hints**
*  For the `axis` parameter of `argmax` and `reduce_max`, if you want to select the **last** axis, one way to do so is to set `axis=-1`.  This is similar to Python array indexing, where you can select the last position of an array using `arrayname[-1]`.
*  Applying `reduce_max` normally collapses the axis for which the maximum is applied.  `keepdims=False` is the default option, and allows that dimension to be removed.  You don't need to keep the last dimension after applying the maximum here.



In [52]:
## UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
## GRADED FUNCTION: yolo_filter_boxes

def yolo_filter_boxes(boxes,  box_confidence, box_class_probs, threshold = .6):
    """ Filters YOLO boxes by thresholding on object and class confidence.
    Filters those  YOLO boxes (Total numebr of YOLO boxes is 19 x 19 x 5) with a class probability 

    Arguments:
      boxes: tensor of shape (19, 19, 5, 4)
      box_confidence: tensor of shape (19, 19, 5, 1)
      box_class_probs: tensor of shape (19, 19, 5, 80)
      threshold: real value, if [max class probability score < threshold], then get rid of the corresponding box

    Returns:
        scores: tensor of shape (?,), containing the class probability score for selected boxes
        boxes: tensor of shape (?, 4), containing (b_x, b_y, b_h, b_w) for selected boxes
        classes: tensor of shape (?,), containing the index of the class detected by the selected boxes

    """
    ## Step 1: Compute box scores
    box_scores = box_confidence * box_class_probs

    ## Step 2: Find the box_classes using the max box_scores, keep track of the corresponding score
    box_classes = tf.math.argmax(box_scores, axis=-1)
    box_class_scores = tf.math.reduce_max(box_scores, axis=-1)

    ## Step 3: Create a filtering mask based on "box_class_scores" by using "threshold". 
    filtering_mask =  box_class_scores >= threshold

    ## Step 4: Apply the mask to scores, boxes and classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)


    return scores, boxes, classes
    

In [54]:
# BEGIN UNIT TEST
tf.random.set_seed(10)
box_confidence = tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
box_class_probs = tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = 0.5)

print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))
print("classes.shape = " + str(classes.shape))



assert type(scores) == EagerTensor, "Use tensorflow functions"
assert type(boxes) == EagerTensor, "Use tensorflow functions"
assert type(classes) == EagerTensor, "Use tensorflow functions"

assert scores.shape == (1789,), "Wrong shape in scores"
assert boxes.shape == (1789, 4), "Wrong shape in boxes"
assert classes.shape == (1789,), "Wrong shape in classes"

assert np.isclose(scores[2].numpy(), 9.270486), "Values are wrong on scores"
assert np.allclose(boxes[2].numpy(), [4.6399336, 3.2303846, 4.431282, -2.202031]), "Values are wrong on boxes"
assert classes[2].numpy() == 8, "Values are wrong on classes"

print("\033[92m All tests passed!")
# END UNIT TEST


scores[2] = 9.270486
boxes[2] = [ 4.6399336  3.2303846  4.431282  -2.202031 ]
classes[2] = 8
scores.shape = (1789,)
boxes.shape = (1789, 4)
classes.shape = (1789,)
 All tests passed!


**Expected Output**:

<table>
    <tr>
        <td>
            <b>scores[2]</b>
        </td>
        <td>
           9.270486
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes[2]</b>
        </td>
        <td>
           [ 4.6399336  3.2303846  4.431282  -2.202031 ]
        </td>
    </tr>
    <tr>
        <td>
            <b>classes[2]</b>
        </td>
        <td>
           8
        </td>
    </tr>
        <tr>
        <td>
            <b>scores.shape</b>
        </td>
        <td>
           (1789,)
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes.shape</b>
        </td>
        <td>
           (1789, 4)
        </td>
    </tr>
    <tr>
        <td>
            <b>classes.shape</b>
        </td>
        <td>
           (1789,)
        </td>
    </tr>

</table>

**Note** In the test for `yolo_filter_boxes`, you're using random numbers to test the function. In real data, the `box_class_probs` would contain non-zero values between 0 and 1 for the probabilities.

The box coordinates in boxes would also be chosen so that heights and widths are non-negative.

